# Hands-On Pertemuan 6: Data Processing dengan Apache Spark

## Tujuan:
- Memahami dan mempraktikkan data processing menggunakan Apache Spark.
- Menggunakan Spark untuk operasi data yang efisien pada dataset besar.
- Menerapkan teknik canggih dalam Spark untuk mengatasi kasus penggunaan nyata.

### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [7]:
# Contoh membuat DataFrame sederhana dan operasi dasar
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('Darmawan', 'CTO', 10000),
        ('DavidCPU', 'HRD', 5100),
        ('MusaSpeaker', 'Enggineer', 2000),
        ('WahyuChipset', 'Marketing', 3900),
        ('AjiGPU', 'Marketing', 4200)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|    Darmawan|       CTO| 10000|
|    DavidCPU|       HRD|  5100|
| MusaSpeaker| Enggineer|  2000|
|WahyuChipset| Marketing|  3900|
|      AjiGPU| Marketing|  4200|
+------------+----------+------+



### 2. Transformasi Dasar dengan DataFrames
Pemrosesan data meliputi transformasi seperti filtering, selections, dan aggregations. Spark menyediakan cara efisien untuk melaksanakan operasi ini.

- **Tugas 2**: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [13]:
# Contoh operasi transformasi DataFrame
df.select('EmployeeName', 'Salary').show()
df.filter(df['Salary'] > 4000).show()
df.groupBy('Department').avg('Salary').show()

from pyspark.sql import functions as F

df.groupBy('Department').agg(
    F.mean('Salary').alias('AverageSalary'),
    F.max('Salary').alias('MaxSalary'),
    F.sum('Salary').alias('TotalSalary')
).show()



+------------+------+
|EmployeeName|Salary|
+------------+------+
|    Darmawan| 10000|
|    DavidCPU|  5100|
| MusaSpeaker|  2000|
|WahyuChipset|  3900|
|      AjiGPU|  4200|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|    Darmawan|       CTO| 10000|
|    DavidCPU|       HRD|  5100|
|      AjiGPU| Marketing|  4200|
+------------+----------+------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|       CTO|    10000.0|
|       HRD|     5100.0|
| Enggineer|     2000.0|
| Marketing|     4050.0|
+----------+-----------+

+----------+-------------+---------+-----------+
|Department|AverageSalary|MaxSalary|TotalSalary|
+----------+-------------+---------+-----------+
|       CTO|      10000.0|    10000|      10000|
|       HRD|       5100.0|     5100|       5100|
| Enggineer|       2000.0|     2000|       2000|
| Marketing|       4050.0|     4200|       8100|
+----------+-------------

### 3. Bekerja dengan Tipe Data Kompleks
Spark mendukung tipe data yang kompleks seperti maps, arrays, dan structs yang memungkinkan operasi yang lebih kompleks pada dataset yang kompleks.

- **Tugas 3**: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [17]:
# Contoh manipulasi tipe data kompleks
df = df.withColumn('SalaryBonus', df['Salary'] * 0.5)
df = df.withColumn('TotalCompensation', df['Salary'] + df['SalaryBonus'])

df.show()

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|    Darmawan|       CTO| 10000|     5000.0|          15000.0|
|    DavidCPU|       HRD|  5100|     2550.0|           7650.0|
| MusaSpeaker| Enggineer|  2000|     1000.0|           3000.0|
|WahyuChipset| Marketing|  3900|     1950.0|           5850.0|
|      AjiGPU| Marketing|  4200|     2100.0|           6300.0|
+------------+----------+------+-----------+-----------------+



### 4. Operasi Data Lanjutan
Menggunakan Spark untuk operasi lanjutan seperti window functions, user-defined functions (UDFs), dan mengoptimalkan query.

- **Tugas 4**: Implementasikan window function untuk menghitung running totals atau rangkings.

In [20]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
windowSpecRank = Window.partitionBy('Department').orderBy(F.desc('Salary'))

df.withColumn('Rank', F.rank().over(windowSpec)).show()

df.withColumn('DenseRank', F.dense_rank().over(windowSpecRank)).show()

df.withColumn('RunningTotal', F.sum('Salary').over(windowSpec)) \
  .withColumn('Rank', F.rank().over(windowSpecRank)) \
  .show()


+------------+----------+------+-----------+-----------------+----+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|Rank|
+------------+----------+------+-----------+-----------------+----+
|    Darmawan|       CTO| 10000|     5000.0|          15000.0|   1|
| MusaSpeaker| Enggineer|  2000|     1000.0|           3000.0|   1|
|    DavidCPU|       HRD|  5100|     2550.0|           7650.0|   1|
|WahyuChipset| Marketing|  3900|     1950.0|           5850.0|   1|
|      AjiGPU| Marketing|  4200|     2100.0|           6300.0|   2|
+------------+----------+------+-----------+-----------------+----+

+------------+----------+------+-----------+-----------------+---------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|DenseRank|
+------------+----------+------+-----------+-----------------+---------+
|    Darmawan|       CTO| 10000|     5000.0|          15000.0|        1|
| MusaSpeaker| Enggineer|  2000|     1000.0|           3000.0|        1|
|    DavidCPU|       H

### 5. Kesimpulan dan Eksplorasi Lebih Lanjut
Review apa yang telah dipelajari tentang pemrosesan data menggunakan Spark dan eksplorasi teknik lebih lanjut untuk mengoptimalkan pemrosesan data Anda.
- **Tugas 5**: Buat ringkasan dari semua operasi yang telah dilakukan dan bagaimana teknik ini dapat diterapkan pada proyek data Anda.